### 验证 Whoosh的工呢过

#### 载入必要的依赖包

In [1]:
import sys,os
sys.path.append("../")
from whoosh.index import create_in, open_dir
from whoosh.fields import *
from whoosh.qparser import QueryParser

from jieba.analyse.analyzer import ChineseAnalyzer

analyzer = ChineseAnalyzer()

#### 定义 SSC 所需要的 Schema

In [4]:
corpus_schema = Schema(id=ID(stored=True), corpus_id=ID(stored=True), field=ID(stored=True), content=TEXT(stored=True, analyzer=analyzer))


if not os.path.exists("corpus_index_dir"):
    os.mkdir("corpus_index_dir")

#### 添加文档

In [5]:
ix = create_in('corpus_index_dir', corpus_schema, 'bot_1')
writer = ix.writer()
writer.add_document(id='123', corpus_id='1', field='faq', content='你们公司在哪里')
writer.add_document(id='124', corpus_id='1', field='faq', content='你们公司在几号线')
writer.commit()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/data/opts/anaconda3/lib/python3.8/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmphghd02cu' -> '/tmp/jieba.cache'
Loading model cost 1.286 seconds.
Prefix dict has been built successfully.


In [11]:
corpus_schema2 = Schema(id=NUMERIC(int, 64, stored=True), corpus_id=NUMERIC(stored=True), field=ID(stored=True), content=TEXT(stored=True, analyzer=analyzer))
ix2 = create_in('corpus_index_dir', corpus_schema, 'bot_2')
writer2 = ix2.writer()
writer2.add_document(id='123', corpus_id='1', field='faq', content='你们公司在哪里')
writer2.add_document(id='124', corpus_id='1', field='faq', content='你们公司在几号线')
writer2.commit()

#### 提交变化

#### 重新加载文档

In [6]:
import whoosh.index as index

ix = index.open_dir('corpus_index_dir', indexname='bot_1')
parser = QueryParser('content', schema=ix.schema)

In [7]:
with ix.searcher() as searcher:
    writer = ix.writer()
    for fields in searcher.all_stored_fields():
        print(fields)
        break


{'content': '你们公司在哪里', 'corpus_id': '1', 'field': 'faq', 'id': '123'}


In [ ]:

with ix.searcher() as s:
    q = parser.parse('公司')
    results = s.search(q)
    for hit in results:
        # print(hit.highlights("content")
        print(hit.get('field'))
    print("==" * 10)

### 测试删除 document

In [17]:

writer1 = ix.writer()
# writer1.delete_by_term('id', '123')
writer1.update_document(id='124', corpus_id='1', field='faq', content='你们公司在几号线!!')

writer1.commit()

In [20]:
ix = index.open_dir('corpus_index_dir', indexname='bot_1')
parser = QueryParser('content', schema=ix.schema)

with ix.searcher() as s:
    q = parser.parse('公司')
    results = s.search(q)
    for hit in results:
        # print(hit.highlights("content")
        print(hit.get('field'), hit.get('content'))
    print("==" * 10)

faq 你们公司在几号线
faq 你们公司在几号线!!
